In [1]:
#my stuff
import icu_data_defs
import transformers
import utils
import features
from constants import column_names,variable_type,clinical_source
import units
import mimic
import logger

#other stuff
from sklearn.model_selection import train_test_split,cross_val_score,ShuffleSplit
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

#make pretty pictures
import seaborn as sns
%matplotlib inline

In [2]:
#HELPER FUNCTIONS

def run_crossval(pipeline,X,y):
    scores_r2 = cross_val_score(pipeline,X,y, scoring='r2',cv=10)
    scores_nmse = cross_val_score(pipeline,X,y, scoring='neg_mean_squared_error',cv=10)

    print 'Cross Validation, K-Fold'
    print 'R^2: {}, {}'.format(scores_r2.mean(),scores_r2.std())
    print 'RMSE: {}, {}'.format(np.sqrt(-1.0*scores_nmse).mean(),np.sqrt(-1.0*scores_nmse).std())

    cv_shuffle = ShuffleSplit(n_splits=10,test_size=0.1)

    scores_r2 = cross_val_score(pipeline,X,y, scoring='r2',cv=cv_shuffle)
    scores_nmse = cross_val_score(pipeline,X,y, scoring='neg_mean_squared_error', cv=cv_shuffle)

    print '\nCross Validation, ShuffleSplit'
    print 'R^2: {}, {}'.format(scores_r2.mean(),scores_r2.std())
    print 'RMSE: {}, {}'.format(np.sqrt(-1.0*scores_nmse).mean(),np.sqrt(-1.0*scores_nmse).std())
    return

"""
Visualize data
"""
#Visualize
def viz_per_feature(df_features,df_labels):  
    plot_cnt = len(df_labels.columns)+1
    
    df_corr = pd.DataFrame(index=df_features.columns,columns=df_labels.columns)
    
    for i,col_name in enumerate(df_features.columns):
        print col_name,'{}/{}'.format(i,df_features.shape[1])
        col = df_features.loc[:,col_name]
        display(col.describe().apply(lambda x: '%.4f' % x).to_frame())
        #determine # of filled values
        mode = col.mode()[0]
        print mode
        mode_count = (col == mode).sum()
        print "MODE:",mode
        print mode_count
        print mode_count/float(col.shape[0])


        # plot histogram of column (all of df_train)
        fig, axarr  = plt.subplots(1,plot_cnt,figsize=(5*(plot_cnt), 5))
        ax = plt.subplot(1, plot_cnt, 1)
        std = col.std()
        mean = col.mean()
        col.loc[(col < (mean + 3.0*std)) & (col > (mean - 3.0*std))].hist()
        ax.set_title('{}_{}\n{}'.format(col_name[0],col_name[1],col_name[2:]))
        ax.set_xlabel(col_name[-2])
        ax.set_ylabel('COUNT')

        #plot this column vs. each label
        for i,label_name in enumerate(df_labels.columns):
            y = df_labels.loc[:,label_name].dropna()
            
            x = col.loc[y.index]
            ax = plt.subplot(1, plot_cnt, 2+i)
            sns.regplot(x, y)
            corr = np.corrcoef(x, y)[0][1]
            ax.set_title('{}_{} vs. {} \n PCC (r) = {}'.format(col_name[0],col_name[1],label_name[0],corr))
            df_corr.loc[col_name,label_name]=corr
            ax.set_xlabel(col_name[-2])
            ax.set_ylabel(label_name)
        
        plt.tight_layout()
        plt.show()
    
    return df_corr
        
"""
Test/train/validate split
"""

def test_train_val_split(all_ids=None,test_size=0.1,random_state=42,print_ids=False):

    if all_ids is None:
        all_ids = mimic.get_all_hadm_ids()
    
    validate_size = test_size/(1-test_size)
    train_size = (1-test_size)*(1-validate_size)
    #these test IDs will never be touched again. They are sacred
    train_val_ids,test_ids = train_test_split(all_ids,test_size=test_size,random_state=random_state)
    train_ids,validate_ids = train_test_split(train_val_ids,test_size=validate_size,random_state=random_state)

    if print_ids:
        print 'Train {}:'.format(int(train_size*100)), len(train_ids),'>',train_ids[:5],'...'
        print 'Validate {}:'.format(int(train_size*100)), len(validate_ids),'>',validate_ids[:5],'...'
        print 'Test {}:'.format(int(test_size*100)), len(test_ids),'>',test_ids[:5],'...'
    return train_ids,validate_ids,test_ids

# Set up

## ETL

In [3]:
# Load Our Data Dict
data_dict = icu_data_defs.data_dictionary('config/data_definitions.xlsx')
display(data_dict.get_defs())

#init ETL Manager => mimic_extract data
etl_fname = 'data/mimic_extract.h5'
etl_manager = mimic.MimicETLManager(etl_fname,'config/mimic_item_map.csv',data_dict)

,component,units,variable_type,clinical_source,lower,upper,list_id
def_id,,,,,,,
0,heart rate,beats/min,qn,observation,0.0,500.0,NaN
1,blood pressure systolic,mmHg,qn,observation,0.0,500.0,NaN
2,blood pressure diastolic,mmHg,qn,observation,0.0,500.0,NaN
3,blood pressure mean,mmHg,qn,observation,0.0,500.0,NaN
4,respiratory rate,insp/min,qn,observation,0.0,150.0,NaN
5,temperature body,degF,qn,observation,0.0,150.0,NaN
6,oxygen saturation pulse oximetry,percent,qn,observation,0.0,100.0,NaN
7,weight body,kg,qn,observation,0.0,700.0,NaN
8,output urine,mL,qn,observation,0.0,30000.0,NaN


In [4]:
etl_manager.etl(components=data_dict.get_components(),save_steps=True) #all components in data dictionary

## Feature Generation

In [5]:
random_state=42
#test/train/val split
train_ids,validate_ids,test_ids = test_train_val_split(print_ids=True,random_state=random_state);

#create all features
m_ureg = units.MedicalUreg()
is_summable = lambda x: m_ureg.is_volume(str(x)) or m_ureg.is_mass(str(x))


"""
Data Specs
"""
summable = {
    column_names.VAR_TYPE : variable_type.QUANTITATIVE,
    column_names.COMPONENT : lambda comp: comp not in  [data_dict.components.WEIGHT_BODY],
    column_names.UNITS: is_summable
}

ordinal = {
    column_names.VAR_TYPE : variable_type.ORDINAL
}

quantitative = {
    column_names.VAR_TYPE : variable_type.QUANTITATIVE
}

nominal = {
    column_names.VAR_TYPE : variable_type.NOMINAL
}

"""
FEATURES
"""

F_mean_qn = features.DataSpecsFeaturizer(
    'mean',
    resample_freq=None,
    data_specs=[quantitative],
    fillna_transformer=Pipeline([
            ('ffill',transformers.GroupbyAndFFill(level=column_names.ID)),
            ('fill_mean',transformers.FillerMean())
        ])
                                     
)

F_mean_ord = features.DataSpecsFeaturizer(
    'mean',
    resample_freq=None,
    data_specs=[ordinal],
    fillna_transformer=Pipeline([
            ('ffill',transformers.GroupbyAndFFill(level=column_names.ID)),
            ('fill_mean',transformers.FillerMode())
        ])
                                     
)

F_last = features.DataSpecsFeaturizer(
    agg_func='last',
    resample_freq=None,
    data_specs=[ordinal,quantitative],
    fillna_transformer=Pipeline([
            ('ffill',transformers.GroupbyAndFFill(level=column_names.ID)),
            ('fill_mean',transformers.FillerMean())
        ])
)


F_std = features.DataSpecsFeaturizer(
    'std',
    resample_freq=None,
    data_specs=[ordinal,quantitative],
    fillna_transformer=transformers.FillerZero()
)

F_sum = features.DataSpecsFeaturizer(
    'sum',
    resample_freq=None,
    data_specs=[summable],
    fillna_transformer=transformers.FillerZero()
)

F_count = features.DataSpecsFeaturizer(
    'count',
    resample_freq=None,
    data_specs=[ordinal,quantitative],
    post_processor = transformers.Replacer(0,np.nan),
    fillna_transformer=transformers.FillerZero()
)

F_count_nom = features.DataSpecsFeaturizer(
    'sum',
    resample_freq=None,
    data_specs=[nominal],
    fillna_transformer=transformers.FillerZero()
)

"""
LABELS
"""
qn_lactate_only={
    column_names.COMPONENT : data_dict.components.LACTATE,
    column_names.VAR_TYPE : variable_type.QUANTITATIVE
}
L_next_lac = features.DataSpecsFeaturizer(
    agg_func='first',
    resample_freq=None,
    data_specs=qn_lactate_only,
    post_processor=transformers.TimeShifter(column_names.DATETIME,shift='infer',n=-1)
)

L_delta_lac = features.DataSpecsFeaturizer(
    agg_func='last',
    resample_freq=None,
    data_specs=qn_lactate_only,
    post_processor=Pipeline([
            ('group_by_id',transformers.ToGroupby(level=column_names.ID)),
            ('delta',transformers.Delta())
        ])
)

Train 80: 47180 > [139698, 127590, 178959, 139276, 196600] ...
Validate 80: 5898 > [112338, 107467, 158733, 144544, 115417] ...
Test 10: 5898 > [167957, 164747, 124147, 184424, 136508] ...


## Smaller Data Set

In [6]:
reload(logger)

train_subset = pd.Series(train_ids).sample(frac=0.2, random_state=random_state).sort_values().tolist()

print train_subset[:5], len(train_subset)

[100014L, 100029L, 100039L, 100046L, 100052L] 9436


In [7]:
reload(features)
#with more memory/a better processor, might not need these first 2 cleaning steps until post-processing
combine_like = Pipeline([
        ('drop_small_columns',transformers.remove_small_columns(threshold=1000)),
        ('drop_low_id_count',transformers.record_threshold(threshold=100)),
        ('combine_like_columns',transformers.combine_like_cols())
    ])

drop_low_counts = Pipeline([
        ('row_threshold',transformers.DropNaN(thresh=20)), #this threshold MAY not apply to a larger feature set.
        ('drop_small_columns',transformers.remove_small_columns(threshold=1000)),
        ('drop_low_id_count',transformers.record_threshold(threshold=100))       
    ])

dsf_labels = features.DataSetFactory(
    featurizers=[
        ('NEXT_LACTATE',L_next_lac),
        ('DELTA_LACTATE',L_delta_lac)
    ],
    resample_freq='2H',
    components=[data_dict.components.LACTATE],
    etl_manager = etl_manager,
    pre_processor = combine_like,
    post_processor = transformers.DropNaN(thresh=1) #drop any rows that have NO labels
)

dsf_features = features.DataSetFactory(
    featurizers=[
        ('MEAN_QN',F_mean_qn),
        ('MEAN_ORD',F_mean_ord),
        ('LAST',F_last),
        ('STD',F_std),
        ('SUM',F_sum),
        ('COUNT',F_count),
        ('COUNT_NOMINAL',F_count_nom),
    ],
    resample_freq='2H',
    components=data_dict.get_components(panel_id=12), # simple data
    etl_manager = etl_manager,
    pre_processor = combine_like,
    post_processor = drop_low_counts

)

In [8]:
df_labels = dsf_labels.fit_transform(train_subset)

(2017-08-17 12:57:36) Make Feature Set. id_count=9436, #features=2
(2017-08-17 12:57:37)>> *fit* Filter columns (remove_small_columns) (28278, 63)
(2017-08-17 12:57:37)<< --- (0.0s)
(2017-08-17 12:57:37)>> *transform* Filter columns (remove_small_columns) (28278, 63)
(2017-08-17 12:57:37)<< --- (0.0s)
(2017-08-17 12:57:37)>> *fit* Filter columns (record_threshold) (28278, 4)
(2017-08-17 12:57:37)<< --- (0.0s)
(2017-08-17 12:57:37)>> *transform* Filter columns (record_threshold) (28278, 4)
(2017-08-17 12:57:37)<< --- (0.0s)
(2017-08-17 12:57:37)>> FIT Combine like columns (28278, 4)
(2017-08-17 12:57:37)>>>> ('lactate', 'known', 'qn', 'mmol/L')
(2017-08-17 12:57:37)<<<< --- (0.0s)
(2017-08-17 12:57:37)<< --- (0.0s)
(2017-08-17 12:57:37)>> TRANSFORM Combine like columns (28278, 4)
(2017-08-17 12:57:37)>>>> ('lactate', 'known', 'qn', 'mmol/L')
(2017-08-17 12:57:38)<<<< --- (1.0s)
(2017-08-17 12:57:38)<< --- (1.0s)
(2017-08-17 12:57:38)>> *fit* Filter columns (DataSpecFilter) (28278, 1)
(2

In [ ]:
df_features = dsf_features.fit_transform(train_subset)

# Models